In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import ensemble
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
rfc = ensemble.RandomForestClassifier()
%matplotlib inline
import time

So here's what you should do. Pick a dataset. It could be one you've worked with before or it could be a new one. Then build the best decision tree you can.

Now try to match that with the simplest random forest you can. For our purposes measure simplicity with runtime. Compare that to the runtime of the decision tree. This is imperfect but just go with it.

In [2]:
# pandas global settings
pd.options.display.max_columns=500
pd.options.display.max_rows=500

In [3]:
path="../../../../Datafiles/titanic/"
file = 'test_complete.csv'
url = 'https://www.kaggle.com/c/titanic/overview'

print("We are working with the Titanic dataset from Kaggle at {}".format(url))
titanic = pd.read_csv(path + file, sep=',', quotechar='"', engine='python')
print("there are {:,} entries in the data frame, with {} columns, from data file: {}".format(len(titanic),
                                                                                             len(titanic.columns),                                                                                            file))
titanic.head(20)
print("titanic has {} rows.".format(len(titanic)))

We are working with the Titanic dataset from Kaggle at https://www.kaggle.com/c/titanic/overview
there are 418 entries in the data frame, with 12 columns, from data file: test_complete.csv
titanic has 418 rows.


In [4]:
# Let's fix some values
titanic["Age"].fillna(titanic["Age"].mean(), inplace=True)
titanic["Fare"].fillna(titanic["Fare"].mean(), inplace=True)

In [5]:
titanic.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000
mean,1100.500000,0.363636,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.481622,0.841838,12.634534,0.896760,0.981429,55.840500
min,892.000000,0.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,0.000000,1.000000,23.000000,0.000000,0.000000,7.895800
50%,1100.500000,0.000000,3.000000,30.272590,0.000000,0.000000,14.454200
75%,1204.750000,1.000000,3.000000,35.750000,1.000000,0.000000,31.500000
max,1309.000000,1.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [6]:
titanic.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
5,897,0,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,NaN,S
6,898,1,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,NaN,Q
7,899,0,2,"Caldwell, Mr. Albert Francis",male,26.0,1,1,248738,29.0000,NaN,S
8,900,1,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,NaN,C
9,901,0,3,"Davies, Mr. John Samuel",male,21.0,2,0,A/4 48871,24.1500,NaN,S


In [34]:
y = titanic['Survived']
print("y is a {}".format(type(y)))
y.columns = ['Survived']
# print(y)
X = titanic.drop(['Survived','Name','Cabin','Ticket'], axis=1) 
X = pd.get_dummies(X)

y is a <class 'pandas.core.series.Series'>


In [36]:
y.head(10)

0    0
1    1
2    0
3    0
4    1
5    0
6    1
7    0
8    1
9    0
Name: Survived, dtype: int64

In [8]:
X.isnull().sum()

PassengerId    0
Pclass         0
Age            0
SibSp          0
Parch          0
Fare           0
Sex_female     0
Sex_male       0
Embarked_C     0
Embarked_Q     0
Embarked_S     0
dtype: int64

In [9]:
X.head(10)

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,892,3,34.5,0,0,7.8292,0,1,0,1,0
1,893,3,47.0,1,0,7.0000,1,0,0,0,1
2,894,2,62.0,0,0,9.6875,0,1,0,1,0
3,895,3,27.0,0,0,8.6625,0,1,0,0,1
4,896,3,22.0,1,1,12.2875,1,0,0,0,1
5,897,3,14.0,0,0,9.2250,0,1,0,0,1
6,898,3,30.0,0,0,7.6292,1,0,0,1,0
7,899,2,26.0,1,1,29.0000,0,1,0,0,1
8,900,3,18.0,0,0,7.2292,1,0,1,0,0
9,901,3,21.0,2,0,24.1500,0,1,0,0,1


In [10]:
X.isnull().sum()

PassengerId    0
Pclass         0
Age            0
SibSp          0
Parch          0
Fare           0
Sex_female     0
Sex_male       0
Embarked_C     0
Embarked_Q     0
Embarked_S     0
dtype: int64

In [11]:
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)  

In [12]:
y_train.sample(20)

320    0
148    0
72     1
218    1
292    0
393    0
136    0
86     1
283    1
163    0
215    0
107    0
38     0
340    0
313    1
382    1
63     1
41     0
321    0
290    0
Name: Survived, dtype: int64

In [39]:
feature_importances = pd.DataFrame(regressor.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)

In [40]:
feature_importances.head(36)*100

,importance
Sex_male,100.0
PassengerId,0.0
Pclass,0.0
Age,0.0
SibSp,0.0
Parch,0.0
Fare,0.0
Sex_female,0.0
Embarked_C,0.0
Embarked_Q,0.0


In [13]:
from sklearn.tree import DecisionTreeRegressor  
regressor = DecisionTreeRegressor()  
regressor.fit(X_train, y_train)  

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [14]:
y_pred = regressor.predict(X_test)  

In [56]:
start_time = time.time()
cross_val_score(regressor, X, y, cv=10)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [57]:
print("Decision Tree elapsed time --- %s seconds ---" % (time.time() - start_time))

Decision Tree elapsed time --- 0.03689289093017578 seconds ---


In [58]:
X.head(10)

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,892,3,34.5,0,0,7.8292,0,1,0,1,0
1,893,3,47.0,1,0,7.0000,1,0,0,0,1
2,894,2,62.0,0,0,9.6875,0,1,0,1,0
3,895,3,27.0,0,0,8.6625,0,1,0,0,1
4,896,3,22.0,1,1,12.2875,1,0,0,0,1
5,897,3,14.0,0,0,9.2250,0,1,0,0,1
6,898,3,30.0,0,0,7.6292,1,0,0,1,0
7,899,2,26.0,1,1,29.0000,0,1,0,0,1
8,900,3,18.0,0,0,7.2292,1,0,1,0,0
9,901,3,21.0,2,0,24.1500,0,1,0,0,1


In [59]:
df=pd.DataFrame({'Actual':y_test, 'Predicted':y_pred})  
df.head(10)

,Actual,Predicted
360,0,0.0
170,0,0.0
224,1,1.0
358,0,0.0
309,1,1.0
308,0,0.0
150,1,1.0
10,0,0.0
21,0,0.0
261,0,0.0


In [60]:
print(y_pred)
from sklearn import metrics  
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))  

[0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 0.
 0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 0. 1. 0. 0.
 0. 0. 0. 1. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1.
 1. 1. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0.]
Mean Absolute Error: 0.0
Mean Squared Error: 0.0
Root Mean Squared Error: 0.0


In [63]:
rfc = ensemble.RandomForestClassifier()

In [64]:
start_time = time.time()
cross_val_score(rfc, X, y, cv=10)

/Users/lou/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/lou/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/lou/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/lou/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/lou/anaconda3/lib/python3.7/site-packages/skl

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [65]:
print("Random Forest elapsed time --- %s seconds ---" % (time.time() - start_time))

Random Forest elapsed time --- 0.11156010627746582 seconds ---
